<a href="https://colab.research.google.com/github/MizunashiPeritec/MyMemo/blob/main/RemoveBackground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像背景削除ライブラリテスト

下記ライブラリを動作させる\
https://github.com/danielgatis/rembg

使い方\
Gitで開いているならタイトルの上にある「Open in Colab」をクリックして\
Google colab で開く

Google Colab で以下のpythonプログラムを上から順に実行する。\
(スクリプトにカーソルを合わせた時の実行マークを押していく)\
一番下のプログラムを実行し、少し待てば背景を削除した画像が表示される

In [ ]:
!pip install rembg

In [ ]:
from google.colab import output
from rembg import remove
import cv2
import base64
from io import BytesIO
from PIL import Image
import numpy as np
import IPython

def run(img):
  try:
    decimg = base64.b64decode(img.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8); 
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    out_img = remove(decimg)

    _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tobytes()
    img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
    return IPython.display.JSON({'result': img_str, 'isCorrect' : True})
  except Exception as e:
    return IPython.display.JSON({'result': e.message, 'isCorrect' : False})

output.register_callback('notebook.run', run)

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(quality=0.8):
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 640 
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      dst_canvasCtx.fillStyle = "black"
      dst_canvasCtx.fillRect(0, 0, dst_canvas.width, dst_canvas.height);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      //debug
      const log = document.createElement('div');
      log.textContent = "test";
      document.body.appendChild(log);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
                        
            if (send_num < 1){
                send_num += 1
                const img = src_canvas.toDataURL(quality);

                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                result.then(value => {
                     
                      parse = JSON.parse(JSON.stringify(value))["data"]
                      parse = JSON.parse(JSON.stringify(parse))["application/json"]

                      isCorrect = JSON.parse(JSON.stringify(parse))["isCorrect"]                      
                      returns = JSON.parse(JSON.stringify(parse))["result"]

                      if(isCorrect) {
                            var image = new Image()
                            image.src = returns
                            image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                            log.textContent = "Correct"
                      } else {
                            dst_canvasCtx.fillStyle = "black"
                            dst_canvas.fillRect(0, 0, dst_canvas.width, dst_canvas.height);
                            log.textContent = returns
                      }

                      send_num -= 1
                })
            }
            
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);   
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  display(js)
  data = eval_js('useCam({})'.format(quality))

In [ ]:
use_cam()